In [2]:
import qrcode

data = """Nama : xxxxx
NIP : 199245732938
Jabatan : Penata muda"""

img = qrcode.make(data)
img.save("namecard_qr.png")


In [6]:
import pandas as pd
import qrcode
from pathlib import Path
import re

# Baca data Excel
df = pd.read_excel("QRCODEdata.xlsx", sheet_name="Sheet1")

# Buat folder output
Path("qr_output").mkdir(exist_ok=True)
def safe_filename(text):
    """Hapus karakter yang tidak aman untuk nama file"""
    text = str(text).strip()           # buang spasi di awal/akhir
    text = text.replace("\n", " ")     # ganti line break jadi spasi
    text = re.sub(r'[<>:"/\\|?*]', '', text)  # buang karakter ilegal
    text = re.sub(r'\s+', '_', text)   # spasi jadi _
    return text

for _, row in df.iterrows():
    # Data yang dimasukkan ke dalam QR
    data = f"""Nama : {row['NAMA']}
NIP : {row['NIP/GOL']}
Jabatan : {row['JABATAN']}"""

    # Buat QR dengan opsi lebih fleksibel
    qr = qrcode.QRCode(
        version=1,
        error_correction=qrcode.constants.ERROR_CORRECT_M,
        box_size=10,
        border=4,
    )
    qr.add_data(data)
    qr.make(fit=True)
    img = qr.make_image(fill_color="black", back_color="white")

    # Bersihkan karakter yang tidak aman untuk nama file
    nama_clean = safe_filename(row["NAMA"])
    jabatan_clean = safe_filename(row["JABATAN"])

    # Nama file: Nama_Jabatan.png
    filename = f"qr_output/{nama_clean}_{jabatan_clean}.png"
    img.save(filename)

print("✅ Semua QR Code sudah tersimpan di folder qr_output")


✅ Semua QR Code sudah tersimpan di folder qr_output


In [16]:
import pandas as pd
import qrcode
from pathlib import Path
from openpyxl import load_workbook
from openpyxl.drawing.image import Image as XLImage
import re

# ==== Konfigurasi ====
excel_path = "QRCODEdata.xlsx"
sheet_name = "Sheet1"
output_folder = Path("qr_output")
output_folder.mkdir(exist_ok=True)

# ==== Fungsi bantu ====
def safe_filename(text):
    text = str(text).strip().replace("\n", " ")
    text = re.sub(r'[<>:\"/\\|?*]', '', text)
    text = re.sub(r'\s+', '_', text)
    return text

def px_to_width(px):  # konversi perkiraan px -> lebar kolom
    return px / 7

def px_to_height(px):  # konversi perkiraan px -> tinggi baris
    return px * 0.75

# ==== Baca Excel ====
df = pd.read_excel(excel_path, sheet_name=sheet_name)

# ==== Generate QR dan simpan file ====
img_paths = []
for _, row in df.iterrows():
    nama = row["NAMA"]
    nip = row["NIP/GOL"]
    jabatan = row["JABATAN"]

    if pd.isna(nama) or pd.isna(jabatan):
        img_paths.append(None)
        continue

    # Teks yang akan dimasukkan ke QR
    qr_text = f"Nama : {nama}\nNIP/GOL : {nip}\nJabatan : {jabatan}"

    # Nama file
    nama_clean = safe_filename(nama)
    jabatan_clean = safe_filename(jabatan)
    file_name = f"{nama_clean}_{jabatan_clean}.png"
    file_path = output_folder / file_name

    # Generate QR
    qr = qrcode.QRCode(
        version=None,
        error_correction=qrcode.constants.ERROR_CORRECT_M,
        box_size=10,   # perbesar kalau mau resolusi tinggi
        border=4,
    )
    qr.add_data(qr_text)
    qr.make(fit=True)
    img = qr.make_image(fill_color="black", back_color="white")
    img.save(file_path)

    img_paths.append(file_path)

# ==== Masukkan gambar ke Excel ====
wb = load_workbook(excel_path)
ws = wb[sheet_name]

col_qr = ws.max_column + 1
ws.cell(row=1, column=col_qr, value="QR_Code")

for idx, path in enumerate(img_paths, start=2):
    if path is None or not path.exists():
        continue
    qr_img = XLImage(str(path))
    w_px, h_px = qr_img.width, qr_img.height

    col_letter = ws.cell(row=1, column=col_qr).column_letter
    ws.column_dimensions[col_letter].width = px_to_width(w_px)
    ws.row_dimensions[idx].height = px_to_height(h_px)

    ws.add_image(qr_img, f"{col_letter}{idx}")

wb.save(excel_path)
print("✅ QR Code berhasil dibuat ulang & dimasukkan ke QRCODEdata.xlsx")


✅ QR Code berhasil dibuat ulang & dimasukkan ke QRCODEdata.xlsx


In [21]:
import pandas as pd
from pathlib import Path
from openpyxl import load_workbook
from openpyxl.drawing.image import Image as XLImage
import re
from barcode import Code128
from barcode.writer import ImageWriter

excel_path = "QRCODEdata2.xlsx"
sheet_name = "Sheet1"
output_folder = Path("barcode_output2")
output_folder.mkdir(exist_ok=True)

def safe_filename(text):
    text = str(text).strip()
    text = re.sub(r'[<>:\"/\\|?*]', '', text)
    text = re.sub(r'\s+', '_', text)
    return text

def px_to_width(px):
    return px / 7

def px_to_height(px):
    return px * 0.75

df = pd.read_excel(excel_path, sheet_name=sheet_name)

wb = load_workbook(excel_path)
ws = wb[sheet_name]
col_bar = ws.max_column + 1
ws.cell(row=1, column=col_bar, value="Barcode")

for i, row in df.iterrows():
    nama = row["NAMA"]
    nip = row["NIP/GOL"]
    jabatan = row["JABATAN"]

    if pd.isna(nama) or pd.isna(jabatan):
        continue

    data = f"Nama = {nama}\nNIP = {nip}\nJabatan = {jabatan}"   # isi yang akan di-encode
    nama_clean = safe_filename(nama)
    jab_clean = safe_filename(jabatan)
    file_name = f"{nama_clean}_{jab_clean}.png"
    file_path = output_folder / file_name

    # Buat barcode tanpa teks
    options = {
        "write_text": False,  # <-- penting
        "module_height": 25,  # tinggi batang
        "quiet_zone": 4
    }
    # Buat barcode
    barcode = Code128(data, writer=ImageWriter())
    barcode.save(str(file_path.with_suffix('')), options)  # pustaka akan otomatis menambah .png

    # Sisipkan ke Excel
    img = XLImage(str(file_path))
    w_px, h_px = img.width, img.height
    col_letter = ws.cell(row=1, column=col_bar).column_letter
    ws.column_dimensions[col_letter].width = px_to_width(w_px)
    ws.row_dimensions[i + 2].height = px_to_height(h_px)
    ws.add_image(img, f"{col_letter}{i + 2}")

wb.save(excel_path)
print("✅ Barcode berhasil dibuat dan dimasukkan ke QRCODEdata.xlsx")


✅ Barcode berhasil dibuat dan dimasukkan ke QRCODEdata.xlsx
